# Distribute Items
Jupyter Notebook to show different patterns for distributing items to another organization.

The patterns can include: [share items](#Share-Item) || [copy & share items](#Copy-then-Share-Item) || [clone items](#Clone-Item) || [clone and modify applications](#Clone-and-Modify-Applications) and more

> ***Note** [Groups](https://doc.arcgis.com/en/arcgis-online/share-maps/groups.htm) can be used to share data so that you maintain ownership without cloning the content. To set up groups, see [configure](https://github.com/mpayson/esri-partner-tools/blob/master/build_org/configure_org.ipynb) for new users, or [create_share](https://github.com/mpayson/esri-partner-tools/blob/master/build_org/create_share_group.ipynb) for existing users.

In [ ]:
from arcgis.gis import GIS, Item

***Note**, if you are unable to import local `partnerutils`, **copy the following functions** from [`clone_utils`](https://github.com/mpayson/esri-partner-tools/blob/master/partnerutils/clone_utils.py)

In [ ]:
from partnerutils.clone_utils import search_group_title, clone_items_modify, search_item_title

In [ ]:
src = GIS(username="mpayson_startups")
tar = GIS(username="mspatialstartups")

#### Example Constants

In [ ]:
GROUP_NAMES = ['Max Test']

SHARE_ITEMID = "90dffd24537240a59eede871ade5856a"
COPY_ITEMID = "90dffd24537240a59eede871ade5856a"
CLONE_ITEMID = "90dffd24537240a59eede871ade5856a"
APP_ID = "1bbda506fdfd4c82a90fea57426f5603"

NAME_TEMPLATE = "{} -- " + tar.properties['urlKey']
FOLDER = "Content -- " + tar.properties['urlKey']

In [ ]:
groups = [search_group_title(target=src, title=n, outside_org=True) for n in GROUP_NAMES]
groups

### Modify Functions
Sometimes you don't want a 1:1 copy, these functions will modify the cloned item properties. A few notes:
* `modify_item_callback` receives the cloned item and its gis. It should return a flattened dict of properties [here](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html?highlight=clone_items#arcgis.gis.Item.update)
* The default behavior is to update the item to match the NAME_TEMPLATE

In [ ]:
def modify_item_callback(item, target_gis):
    title = NAME_TEMPLATE.format(item.title)
    while search_item_title(target_gis, title):
        title = input("Title `{0}` for ITEM `{1}` already exists \n new title: "
                         .format(title, item.title))
    return {"title": title}

## Execution

### Share Item
Shares an item with an existing group. Use this pattern if you want to own and maintain single authoritative items that all of your user organizations can access.

In [ ]:
share_item = Item(src, SHARE_ITEMID)

In [ ]:
share_item.share(groups=groups)

### Copy then Share Item
Copies and shares an item to an existing group. Use this pattern if you have a template item and you want each user organization to access a copy while **you own and maintain the copies**.

In [ ]:
copy_item = Item(src, COPY_ITEMID)

In [ ]:
paste_items = clone_items_modify([copy_item], src,
                               modify_item_callback=modify_item_callback,
                               copy_data=False, search_existing_items=False,
                               folder=FOLDER)
paste_share_res = [i.share(groups=groups) for i in paste_items]
paste_share_res

### Clone Item
Clones an item to a target organization. Use this pattern if you have a template item and you want **each user organization to own their copy**.

In [ ]:
clone_item = Item(src, CLONE_ITEMID)

In [ ]:
clones = clone_items_modify([clone_item], tar,
                               modify_item_callback=modify_item_callback,
                               copy_data=True, search_existing_items=False,
                               folder=FOLDER)
clones

### Clone and Modify Applications
Clones an application (or webmap) then swizzles the dependencies. Use this pattern if you have a template application/map that should be configured for data that your user organizations can now access.

By default, the clone_items functions will also clone dependencies. This example shows how to reference the newly shared item via a group, so that the end-organization owns the application but you maintain the underlying data.

In [ ]:
app_item = Item(src, APP_ID)
item_map = {copy_item.id: paste_items[0].id}
item_map

In [ ]:
app_clones = clone_items_modify([app_item], tar,
                               modify_item_callback=modify_item_callback,
                               copy_data=False, search_existing_items=False,
                               folder=FOLDER,
                               item_mapping=item_map)
app_clones